# Train/Fine Tune Tesseract model

### Import libraries

In [1]:
import os
import py7zr
import subprocess
from utilities import *
from evaluate_model import *
from generate_output_text import process_images
from generate_word_char_error import generate_errors

### Install Tesseract

#### Install needed libraries

In [ ]:
%pip install Pillow>=6.2.1 python-bidi>=0.4 matplotlib pandas

#### Install Tesseract and needed packages

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!sudo apt install bc

#### Clone Tesseract

In [ ]:
!git clone https://github.com/tesseract-ocr/tesseract.git

#### Install make

In [ ]:
cd tessract

In [ ]:
!./autogen.sh

In [ ]:
!./configure

In [ ]:
!make

In [ ]:
!sudo make install

In [ ]:
!sudo ldconfig

In [ ]:
!make training

In [ ]:
!sudo make training-install

In [ ]:
cd ..

### Clone tesstrain and tessdata_best

In [1]:
!git clone https://github.com/tesseract-ocr/tesstrain.git

Cloning into 'tesstrain'...
remote: Enumerating objects: 997, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 997 (delta 130), reused 208 (delta 127), pack-reused 775
Receiving objects: 100% (997/997), 13.41 MiB | 3.18 MiB/s, done.
Resolving deltas: 100% (572/572), done.


In [5]:
!git config --global http.version HTTP/1.1

In [2]:
!git clone https://github.com/tesseract-ocr/tessdata_best.git -j12

Cloning into 'tessdata_best'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 257 (delta 1), reused 4 (delta 0), pack-reused 250
Receiving objects: 100% (257/257), 1.30 GiB | 2.05 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Updating files: 100% (168/168), done.


### Create langdata folder

In [3]:
cd tesstrain

/Users/joe/Documents/ocr/tesstrain


/Users/joe/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Install required data

In [5]:
%pip install -r requirements.txt
!make tesseract-langdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 3.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
wget -O data/langdata/radical-stroke.txt 'https://github.com/tesseract-ocr/langdata_lstm/raw/main/radical-stroke.txt'
make: wget: No such file or directory
make: *** [data/langdata/radical-stroke.txt] Error 1


### Create the needed folders for the model

In [ ]:
!mkdir ./data/ara
!mkdir ./data/Tesseract_F_JSTOR_TRDG
!mkdir ./data/Tesseract_F_JSTOR_TRDG-ground-truth

In [7]:
cd ..

/Users/joe/Documents/ocr


/Users/joe/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Make a copy of all files in the needed_files folders to the model folder in tesstrain

In [ ]:
folder_cp_path = './needed_files/ara'
folder_paste_path = './tesstrain/data/Tesseract_F_JSTOR_TRDG'
for file in os.listdir(folder_cp_path):
    file_path = os.path.join(folder_cp_path, file)
    subprocess.run(f'cp {file_path} {folder_paste_path}', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [12]:
folder_cp_path = './needed_files/rest_data'
folder_paste_path = './tesstrain/data'
for file in os.listdir(folder_cp_path):
    file_path = os.path.join(folder_cp_path, file)
    subprocess.run(f'cp {file_path} {folder_paste_path}', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

### Rename files in the model folder

In [ ]:
model_name = 'Tesseract_F_JSTOR_TRDG'
folder_path = './tesstrain/data/Tesseract_F_JSTOR_TRDG'
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    file_name_split = file.split('.')
    new_name_path = os.path.join(folder_path, model_name + '.' + file_name_split[1])
    subprocess.run(f'mv {file_path} {new_name_path}', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

### Unzip all datasets inside the ground truth

In [ ]:
unzip_folder_path = './tesstrain/data/Tesseract_F_JSTOR_TRDG-ground-truth'
datasets_path = './datasets'
for ds in os.listdir(datasets_path):
    ds_path = os.path.join(datasets_path, ds)
    with py7zr.SevenZipFile(ds_path, mode='r') as z:
        z.extractall(path=unzip_folder_path)

#### Copy all files from every dataset folder to the ground truth

In [ ]:
ground_truth_path = './tesstrain/data/Tesseract_F_JSTOR_TRDG-ground-truth'
datasets_folders = ['Blur_1p8', 'Distortion_T2', 'Distortion_T_3', 'Dist_Type2', 'Dist_Type_0', 'JSTORArabic-ground-truth', 'num_dataset_FINAL', 'outputbase', 'Rand_Skew_Ang3', 'rand_dataset_txt2img']

for dir in datasets_folders:
    dir_path = os.path.join(ground_truth_path, dir)
    subprocess.run(f'cp -a {dir_path}/. {ground_truth_path}/', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f'rm -fr {dir_path}', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

subprocess.run(f'rm -fr ./tesstrain/data/Tesseract_F_JSTOR_TRDG-ground-truth/.ipynb_checkpoints', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

#### Train/Fine Tune the model

In [ ]:
cd tesstrain

need to change LEARNING_RATE, MAX_ITERATIONS and EPOCHS values

In [ ]:
!make LANG_TYPE=RTL MODEL_NAME=Tesseract_F_JSTOR_TRDG PSM=13 START_MODEL=ara TESSDATA=../tessdata_best MAX_ITERATIONS=500 RATIO_TRAIN=0.80 EPOCHS=500 training -j12

In [ ]:
cd ..